---
title: "Computational Methods in Economics - Problem Set 3"
author: "Ricardo Semião e Castro"
format: 
    html:
        embed-resources: true
---

Instructions in [ps3_instructions.pdf](ps3/ps3_instrkctions.pdf). This project counts with a [Manifest.toml](Manifest.toml) and [Project.toml](Project.toml) files, generated via [config.jl](config.jl). These will not be included in the final submission. The HTML file is generated via Quarto.

I use LLM models to speed up coding, but only via inline suggestions, not to create whole chunks of code. Additionally, I use them to answer questions about the Julia language, that I had no contact before this class.

In [ ]:
#| output: false

using FastGaussQuadrature
using Printf
using Random, Statistics
using DataStructures

Random.seed!(101134385455)

TaskLocalRNG()

As many questions will ask for trying several times a approximation method, I will define a custom function for it, `approx_fun_n`.

It recieves:

- A function to approximate, `f`.
- An algorithm (function) to approximate `f` with, `a`.
- And `n`, some parameter of `a` that controls the precision of the approximation, often, a number of iterations or samples to be done.

`approx_fun_n` assumes that `a` has the first argument `f` and second `n`. In order to enforce that, I will have it set as a custom type `Approximator`, whose call method requires that order.

In [ ]:
#| output: false

struct Approximator
    approx::Function
end

function (a::Approximator)(f::Function, n::Int64; args...)
    return a.approx(f, n; args...)
end

In [ ]:
#| output: false

function approx_fun_n(
    f::Function, a::Approximator, ns::Tuple,
    text::String = "f(.)", fmt::String = "%+.2e";
    kwargs...
)
    output = OrderedDict{String, String}()

    for n in ns
        result = Printf.format(Printf.Format(fmt), a(f, n; kwargs...))
        output["n = $n"] = "$text ≈ $result"
    end

    return output
end

approx_fun_n (generic function with 3 methods)

# Question 1.

Let $X \sim N(0, 1)$.

## Item a)

We know that the Gauss-Hermite quadrature approximates integrals of the form:

$$
\int^{\infty}_{-\infty}f(x) \cdot e^{-x^2} ~dx
$$

Thus, we will write the integral that defines the average of a normal RV, then apply variable substitution to make it fit the Gauss-Hermite quadrature. Let $\phi$ be the Gaussian PDF.

$$
E[X] = \int^{\infty}_{-\infty}x \cdot \phi(x) ~dx = \int^{\infty}_{-\infty}x \cdot \frac{1}{\sqrt{2\pi}} e^{-x^2/2} ~dx
$$

Let $y = x/\sqrt{2}$, then $dy = dx/\sqrt{2}$, and:

$$
E[X] = \int^{\infty}_{-\infty}\sqrt{2}y \cdot \frac{1}{\sqrt{2\pi}} e^{-(\sqrt{2}y)^2/2} ~\sqrt{2}dy =
$$
$$
\int^{\infty}_{-\infty}y \cdot \frac{1}{\sqrt{\pi}} e^{-y^2} ~\sqrt{2}dy = \frac{\sqrt{2}}{\sqrt{\pi}}\int^{\infty}_{-\infty}y \cdot e^{-y^2} ~dy
$$

We can see the desired format, with $f(x) = x$. Thus, we have that:

$$
\mathbb{E}[X] \approx \frac{\sqrt{2}}{\sqrt{\pi}} \sum_{i=1}^{n} w_i \cdot x_i
$$

Then, we can create the optimal values for $x_i$ and $w_i$ for each $n$ via `gausshermite()`, and get our results.

In [ ]:
#| output: false

approx_gausshermite = Approximator(
    function (f, n; c)
        x, w = gausshermite(n)
        return c * sum(f.(x) .* w)
    end
)

Approximator(var"#13#15"())

Now, lets run the approximation for the requested `n` values:

In [5]:
ns = (3, 5, 10)
f = x::Float64 -> x

approx_fun_n(f, approx_gausshermite, ns, "E[X]"; c = (sqrt(2)/sqrt(pi)))

OrderedDict{String, String} with 3 entries:
  "n = 3"  => "E[X] ≈ -8.42e-16"
  "n = 5"  => "E[X] ≈ -6.64e-16"
  "n = 10" => "E[X] ≈ -1.48e-17"

We can see that the results are indeed very close to $0$, as expected. The values get closer the bigger the $n$, also as expected.

## Item b)

For the Monte Carlo integration, we will sample $X$ from its distribution and calculate the average.

In [ ]:
#| output: false

approx_montecarlo = Approximator(
    function (f, n)
        return mean(f(n))
    end
)

Approximator(var"#18#19"())

In [7]:
ns = 10 .^ (2, 4, 6)
f = randn

approx_fun_n(f, approx_montecarlo, ns, "E[X]")

OrderedDict{String, String} with 3 entries:
  "n = 100"     => "E[X] ≈ +3.41e-02"
  "n = 10000"   => "E[X] ≈ -6.08e-03"
  "n = 1000000" => "E[X] ≈ +6.70e-05"

## Item c)

I'll start by breaking the expectation into the combination of the conditionals:

$$
E[\max(1, X)] = 1 \cdot P(X < 1) + E[X | X > 1] \cdot P(X > 1)
$$
$$
E[\max(1, X)] = \Phi(1) + \frac{\phi(1)}{1 - \Phi(1)} (1 - \Phi(1))
$$
$$
E[\max(1, X)] = \Phi(1) + \phi(1)\\
$$

To understand the passage of $E[X | X > 1]$, note that $X | X > a$ is a truncated variable. A truncated distribution of $f_X$ is one $f_{X^*}$ where:

$$
f_{X^*}(x) \coloneqq f_X(x \mid X \geq a) =\begin{cases}
0, & x < a \\
\frac{f_X(x)}{1 - F_X(a)}, & x \geq a
\end{cases}
$$
$$
E[X \mid X \geq a] \coloneqq \frac{\int_{a}^\infty x f_X(x) ~dx}{1 - F_X(a)}
$$

And the top integral can be solved via substitution ($u = y^2/2$) as $\phi(a)$.

Using Gaussian tables, we get:

- $\Phi(1) \approx 0.8413$
- $\phi(1) \approx 0.2419$

Then:

$$
E[\max(1, X)] \approx 0.8413 + 0.2419 = 1.0832
$$

## Item d)

The integral we want to approximate is:

$$
E[\max(1, X)] = \int^\infty_{-\infty} \max(1, X) \cdot \frac{1}{\sqrt{2\pi}} e^{-x^2/2} ~dx
$$

Lets apply the same substitution as before:

$$
E[\max(1, X)] = \int^\infty_{-\infty} \max(1, x) \cdot \frac{1}{\sqrt{2\pi}} e^{-x^2/2} ~dx
$$
$$
E[\max(1, X)] = \int^{\infty}_{-\infty}\max(1, \sqrt{2}y) \cdot \frac{1}{\sqrt{2\pi}} e^{-(\sqrt{2}y)^2/2} ~\sqrt{2}dy
$$
$$
E[\max(1, X)] = \frac{1}{\sqrt{\pi}} \int^{\infty}_{-\infty}\max(1,\sqrt{2}y) \cdot e^{-y^2} ~dy
$$

We can see the desired format, with $f(x) = \max(1,\sqrt{2}x)$. Thus, we have that:

$$
\mathbb{E}[X] \approx \frac{1}{\sqrt{\pi}} \sum_{i=1}^{n} w_i \cdot \max(1,\sqrt{2}x_i)
$$

Then, we can create the optimal values for $x_i$ and $w_i$ for each $n$ via `gausshermite()`, and get our results.

In [8]:
ns = (3, 5, 10)
f = x::Float64 -> max(sqrt(2)*x, 1)

approx_fun_n(f, approx_gausshermite, ns, "E[X]", "%.4f"; c = (1/sqrt(pi)))

OrderedDict{String, String} with 3 entries:
  "n = 3"  => "E[X] ≈ 1.1220"
  "n = 5"  => "E[X] ≈ 1.0999"
  "n = 10" => "E[X] ≈ 1.0935"

For the Monte Carlo integration, we will sample $X$ from its distribution and calculate the average of $\max(1,X)$.

In [9]:
ns = 10 .^ (2, 4, 6)
f = n::Int64 -> max.(randn(n), 1)

approx_fun_n(f, approx_montecarlo, ns, "E[X]", "%.4f")

OrderedDict{String, String} with 3 entries:
  "n = 100"     => "E[X] ≈ 1.1389"
  "n = 10000"   => "E[X] ≈ 1.0829"
  "n = 1000000" => "E[X] ≈ 1.0833"

Both results are as expected, similar to the analytical one, and approaching it as $n$ increases.

# Question 2.

## Item a)

Lets start by writing $u$'s distribution, a independent joint normal distribution:

$$
E[\max(X,Y)] = \int_{-\infty}^\infty \int_{-\infty}^\infty \max(x,y) \cdot \frac{1}{2\pi} \exp\left( -\frac{x^2 + y^2}{2} \right) ~dx ~dy
$$

Similarlly to before, we will aplly the substitutions:

$$
x = \sqrt{2}k, ~~ y = \sqrt{2}v
$$
$$
dx = \sqrt{2}dk, ~~ dy = \sqrt{2}dv 
$$

Then:

$$
\exp\left(-\frac{x^2 + y^2}{2} \right) = \exp\left( -\frac{2k^2 + 2v^2}{2} \right) = \exp(-k^2 - v^2)
$$
$$
E[\max(X,Y)] = \frac{1}{\pi} \int_{-\infty}^\infty \int_{-\infty}^\infty \max(\sqrt{2}k, \sqrt{2}v) \cdot e^{-k^2} e^{-v^2} ~dk ~dv
$$

Which is the desired format, with $f(k,v) = \max(\sqrt{2}k, \sqrt{2}v)$. Thus, we have that:

$$
E[\max(X,Y)] \approx \frac{1}{\pi} \sum_{i=1}^n\sum_{j=1}^n w^x_i w^y_j \cdot \max(\sqrt{2}x_i, \sqrt{2}y_i)
$$

In [10]:
n = 10
q2a_quad_result = 0.0

f = (x::Float64, y::Float64) -> max(sqrt(2)*x, sqrt(2)*y)
x, w = gausshermite(n)

for i in 1:n
    for j in 1:n
        q2a_quad_result += w[i] * w[j] * f(x[i], x[j])
    end
end

q2a_quad_result = q2a_quad_result / pi

"n = $n => E[X] ≈ $q2a_quad_result"

"n = 10 => E[X] ≈ 0.5412942646726828"

## Item b)

For the Monte Carlo integration, we will independently sample $X$ and $Y$ from their distribution, and calculate the average of $\max(X,Y)$.

In [11]:
n = 10^6

q2b_quad_result = mean(max.(randn(n), randn(n)))

"n = $n => E[X] ≈ $q2b_quad_result"

"n = 1000000 => E[X] ≈ 0.5648290470049241"

# Question 3.

First, lets write code for the trapezoid rule:

$$
\int_{a}^{b}\,f(x)d x \approx \sum_{i=1}^{n}\,\frac{h}{2}\,(f(x_{i})+f(x_{i+1}))
$$

Lets use $n$ (user-defined) equally spaced points for the summation.

In [ ]:
#| output: false

approx_newton_trapezoid = Approximator(
    function (f::Function, n::Int64; lims::Tuple{Float64, Float64})
        a, b = lims
        h = (b - a) / n
        x = a:h:b
        return (h/2) * (f(a) + f(b) + 2*sum(f.(x[2:end-1])))
    end
)

Approximator(var"#26#28"())

## Item a)

In [13]:
ns = (3, 5, 10, 15, 20)
f = x::Float64 -> x

approx_fun_n(f, approx_newton_trapezoid, ns, "E[X]", "%.4f"; lims = (0.0, 1.0))

OrderedDict{String, String} with 5 entries:
  "n = 3"  => "E[X] ≈ 0.5000"
  "n = 5"  => "E[X] ≈ 0.5000"
  "n = 10" => "E[X] ≈ 0.5000"
  "n = 15" => "E[X] ≈ 0.5000"
  "n = 20" => "E[X] ≈ 0.5000"

The trapezoid rule uses a degree 1 polinomial to approximate the function. The function $f(x) = x$ is a degree 1 polinomial, so it makes sense that we get a perfect approximation. The value $0.5$ can be thought of as the area of the triangle that relates to the integral.

## Item b)

In [14]:
ns = (3, 5, 10, 15, 20)
f = x::Float64 -> x * sin(x)

approx_fun_n(f, approx_newton_trapezoid, ns, "E[X]", "%.4f"; lims = (0.0, 1.0))

OrderedDict{String, String} with 5 entries:
  "n = 3"  => "E[X] ≈ 0.3140"
  "n = 5"  => "E[X] ≈ 0.3058"
  "n = 10" => "E[X] ≈ 0.3023"
  "n = 15" => "E[X] ≈ 0.3017"
  "n = 20" => "E[X] ≈ 0.3015"

We don't have a perfect match anymore, but it is converging, since:

$$
\int^1_0 x \sin(x) ~dx = -x\cos(x) + \sin(x)\big|^1_0 \approx 0.301168
$$

## Item c)

In [15]:
ns = (3, 5, 10, 15, 20)
f = x::Float64 -> sqrt(1 - x^2)

approx_fun_n(f, approx_newton_trapezoid, ns, "E[X]", "%.4f"; lims = (0.0, 1.0))

OrderedDict{String, String} with 5 entries:
  "n = 3"  => "E[X] ≈ 0.7294"
  "n = 5"  => "E[X] ≈ 0.7593"
  "n = 10" => "E[X] ≈ 0.7761"
  "n = 15" => "E[X] ≈ 0.7803"
  "n = 20" => "E[X] ≈ 0.7821"

Again, we don't have a perfect match, but it is converging, since we have one fourth of the area of a unit circle:

$$
\int^1_0 \sqrt{1 - x^2} ~dx = \frac{1}{4} \pi 1^2 \approx 0.78539
$$

# Question 4.

First, note the true derivatives:

$$
f(x) = x^2 \implies f'(x) = 2x \implies f'(5) = 10
$$
$$
f(x) = \log(x) \implies f'(x) = \frac{1}{x} \implies f'(10) = 0.1
$$
$$
f(x) = x\sin(x) \implies f'(x) = \sin(x) + x\cos(x) \implies f'(1) \approx 1.38177\\
$$

Now, lets define the difference functions:

In [ ]:
#| output: false

approx_diff_list = (
    p2 = (f, x, h) -> (f(x + h) - f(x - h)) / (2*h),
    p4 = (f, x, h) -> (-f(x + 2*h) + 8*f(x + h) - 8*f(x - h) + f(x - 2*h)) / (12*h)
)

hs = (0.001, 0.005, 0.01, 0.05)

(0.001, 0.005, 0.01, 0.05)

And a custom function to apply them:

In [ ]:
#| output: false

function approx_fun_diff(f::Function, p::Float64, hs::Tuple, diff_list::NamedTuple, comparison::Float64, fmt::String = "%+.3e")
    output = OrderedDict{String, String}()

    for (key, diff) in pairs(diff_list)
        for h in hs
            result = Printf.format(Printf.Format(fmt), diff(f, p, h) - comparison)
            output["points = $key, h = $h"] = "f'($p) ≈ $result"
        end
    end

    return output
end

approx_fun_diff (generic function with 2 methods)

## Item a)

In [18]:
f = x::Float64 -> x^2
p = 5.0

results_q4_x2 = approx_fun_diff(f, p, hs, approx_diff_list, 10.0)

OrderedDict{String, String} with 8 entries:
  "points = p2, h = 0.001" => "f'(5.0) ≈ +3.340e-12"
  "points = p2, h = 0.005" => "f'(5.0) ≈ -2.132e-13"
  "points = p2, h = 0.01"  => "f'(5.0) ≈ -2.132e-13"
  "points = p2, h = 0.05"  => "f'(5.0) ≈ -3.553e-14"
  "points = p4, h = 0.001" => "f'(5.0) ≈ +4.524e-12"
  "points = p4, h = 0.005" => "f'(5.0) ≈ -2.132e-13"
  "points = p4, h = 0.01"  => "f'(5.0) ≈ -2.718e-13"
  "points = p4, h = 0.05"  => "f'(5.0) ≈ -2.487e-14"

All the approximations are near perfect, with the smallest being with 4 points and $h = 0.5$.

## Item b)

In [19]:
f = x::Float64 -> log(x)
p = 10.0

results_q4_x2 = approx_fun_diff(f, p, hs, approx_diff_list, 0.1)

OrderedDict{String, String} with 8 entries:
  "points = p2, h = 0.001" => "f'(10.0) ≈ +3.333e-10"
  "points = p2, h = 0.005" => "f'(10.0) ≈ +8.333e-09"
  "points = p2, h = 0.01"  => "f'(10.0) ≈ +3.333e-08"
  "points = p2, h = 0.05"  => "f'(10.0) ≈ +8.333e-07"
  "points = p4, h = 0.001" => "f'(10.0) ≈ -1.591e-13"
  "points = p4, h = 0.005" => "f'(10.0) ≈ -4.062e-14"
  "points = p4, h = 0.01"  => "f'(10.0) ≈ -7.393e-14"
  "points = p4, h = 0.05"  => "f'(10.0) ≈ -5.000e-11"

The approximations are worse than before, but still good. The smallest error is with 4 points and $h = 0.005$.

## Item c)

In [20]:
f = x::Float64 -> x * sin(x)
p = 1.0

results_q4_x2 = approx_fun_diff(f, p, hs, approx_diff_list, 1.38177)

OrderedDict{String, String} with 8 entries:
  "points = p2, h = 0.001" => "f'(1.0) ≈ +2.780e-06"
  "points = p2, h = 0.005" => "f'(1.0) ≈ -9.479e-06"
  "points = p2, h = 0.01"  => "f'(1.0) ≈ -4.779e-05"
  "points = p2, h = 0.05"  => "f'(1.0) ≈ -1.273e-03"
  "points = p4, h = 0.001" => "f'(1.0) ≈ +3.291e-06"
  "points = p4, h = 0.005" => "f'(1.0) ≈ +3.291e-06"
  "points = p4, h = 0.01"  => "f'(1.0) ≈ +3.289e-06"
  "points = p4, h = 0.05"  => "f'(1.0) ≈ +2.302e-06"

The results are again worse, given this highly non-linear function. The smallest error is with 4 points and $h = 0.005$.

## Item d)

Already answered before.